# Analysis of MNIST dataset with tensorflow 

In [ ]:
#import the relevan libraries
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

In [ ]:
#load the data
mnist_dataset, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)

In [ ]:
mnist_info

In [ ]:
#set the sizes of the test data
test_size = mnist_info.splits['test'].num_examples
test_size

In [ ]:
#set the size of the training data
train_size = mnist_info.splits['train'].num_examples
train_size

In [ ]:
#set the size of the validation data
validation_size = tf.cast(0.1 * train_size, tf.int64)
validation_size

In [ ]:
#save the train data
train_validation_data = mnist_dataset['train']

In [ ]:
#save the test data
test_mnist_data = mnist_dataset['test']

In [ ]:
# image matrices take integer values ranging from 0 to 255
# write a function to scale it so that they range from 0 to 1

def scale_image(image, label):
    
    #convert integer values to float
    image = tf.cast(image, tf.float32)
    
    #scale the images
    image = image / 255
    
    #return the values
    return image, label

In [ ]:
#scale the train data
scaled_train_validation_data = train_validation_data.map(scale_image)

In [ ]:
#scale the test data
scaled_test_data = test_mnist_data.map(scale_image)

In [ ]:
#set the BUFFER SIZE for shuffling large data sets
buffer_size = 10000

#shuffle the train data (no need to shuffle test_data)
shuffled_scaled_train_validation_data = scaled_train_validation_data.shuffle(buffer_size = buffer_size)

In [ ]:
#take the validation data
shuffled_scaled_validation_data = shuffled_scaled_train_validation_data.take(validation_size)

In [ ]:
#take the train data
shuffled_scaled_train_data = shuffled_scaled_train_validation_data.skip(validation_size)

In [ ]:
#set the batch size for SGD (stochastic gradient descent)
batch = 100

In [ ]:
#batch the training data
train_data = shuffled_scaled_train_data.batch(batch_size = batch)

In [ ]:
#batch the validation data
validation_data = shuffled_scaled_validation_data.batch(validation_size)

In [ ]:
#batch the test data
test_data = scaled_test_data.batch(test_size)

In [ ]:
#seperate validation inputs and outputs
validation_inputs, validation_outputs = next(iter(validation_data))

# Model

In [ ]:
#determine the input size (each image is 28x28 pixels)
input_size = 28 * 28

In [ ]:
#determine the number of outputs (to one-hot-code a digit we need ten outputs)
output_size = 10

In [ ]:
#define the hidden layer size
#traditionally all hidden layers have the same size
hidden_layer_size = 50

In [ ]:
#define the model as sequential layers of inputs, hidden layers and outputs
model = tf.keras.Sequential([
                               #flatten the (28X28X1) tensor of rank 3 to (784,) or feed it to CNN 
                               tf.keras.layers.Flatten(input_shape=(28,28,1)),
                                
                               #stack the hidden layers and apply activation functions 
                               tf.keras.layers.Dense(hidden_layer_size, activation='relu' ),
    
                               #stack another hidden layer and apply the activation function
                               tf.keras.layers.Dense(hidden_layer_size, activation='relu' ),
                    
                               #apply as many hidden layer as needed       
                                
                               #stack final layer and apply the softmax activation function to return 1s and 0s
                               tf.keras.layers.Dense(output_size, activation='softmax' )    
])

# Optimize

In [ ]:
#choose the optimizer and the loss function
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train

In [ ]:
# randomly pick the number of epochs
num_epochs = 5

In [ ]:
#fit the data
model.fit(train_data, epochs=num_epochs, validation_data= (validation_inputs, validation_outputs ), 
          verbose=2, validation_steps=1)

# Test

In [ ]:
test_loss, test_accuracy = model.evaluate(test_data)

In [ ]:
print ('Test loss is {:.2f} and test accuracy is {:.2f}%'.format(test_loss, test_accuracy*100))

## This model is able to predict 98% of the hand written digits correctly. 